In [1]:
import ee
import colorcet
import geemap
from datetime import datetime, timedelta
import numpy as np
import wxee

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='hotspotstoplight')

In [4]:
bbox = ee.Geometry.Polygon([
    [
        [-85.9, 8.0],  # Lower left corner (southwest), moved south
        [-85.9, 11.2], # Upper left corner (northwest), moved north
        [-82.5, 11.2], # Upper right corner (northeast), moved north
        [-82.5, 8.0],  # Lower right corner (southeast), moved south
        [-85.9, 8.0]   # Closing the polygon by repeating the first point, moved south
    ]
])

start_date = '2021-07-22'
end_date = '2021-07-28'
start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")

year_before_start = start_date - timedelta(days=365)
start_of_year = datetime(year_before_start.year, 1, 1)
end_of_year = datetime(year_before_start.year, 12, 31)

In [5]:
# Load the datasets
dem = ee.Image('WWF/HydroSHEDS/03VFDEM').clip(bbox)
slope = ee.Terrain.slope(dem)
landcover = ee.Image("ESA/WorldCover/v100/2020").select('Map').clip(bbox)
flow_direction = ee.Image('WWF/HydroSHEDS/03DIR').clip(bbox)
ghsl = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").clip(bbox)

In [6]:
stream_dist_proximity_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/stream-outlet-distance/stream_dist_proximity")\
    .filterBounds(bbox)\
    .mosaic()
stream_dist_proximity = stream_dist_proximity_collection.clip(bbox).rename('stream_distance')

flow_accumulation_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/base-network-layers/flow_accumulation")\
    .filterBounds(bbox)\
    .mosaic()
flow_accumulation = flow_accumulation_collection.clip(bbox).rename('flow_accumulation')

spi_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/spi")\
    .filterBounds(bbox)\
    .mosaic()
spi = spi_collection.clip(bbox).rename('spi')

sti_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/sti")\
    .filterBounds(bbox)\
    .mosaic()
sti = sti_collection.clip(bbox).rename('sti')

cti_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/cti")\
    .filterBounds(bbox)\
    .mosaic()
cti = cti_collection.clip(bbox).rename('cti')

tpi_collection = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tpi")\
    .filterBounds(bbox)\
    .mosaic()
tpi = tpi_collection.clip(bbox).rename('tpi')

tri_collection = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tri")\
    .filterBounds(bbox)\
    .mosaic()
tri = tri_collection.clip(bbox).rename('tri')

pcurv_collection = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/pcurv")\
    .filterBounds(bbox)\
    .mosaic()
pcurv = pcurv_collection.clip(bbox).rename('pcurv')

tcurv_collection = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tcurv")\
    .filterBounds(bbox)\
    .mosaic()
tcurv = tcurv_collection.clip(bbox).rename('tcurv')

aspect_collection = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/aspect")\
    .filterBounds(bbox)\
    .mosaic()
aspect = aspect_collection.clip(bbox).rename('aspect')

In [7]:
# Load the precipitation data.
precipitation_data = ee.ImageCollection("NASA/GDDP-CMIP6") \
    .filterBounds(bbox) \
    .filterDate(start_of_year, end_of_year) \
    .select('pr') \
    .filter(ee.Filter.eq('model', 'ACCESS-CM2'))

# Function to compute max precipitation for a given month and assign a unique band name.
def get_max_precip_for_month(month):
    # Filter the collection for the given month.
    filtered_month = precipitation_data.filter(ee.Filter.calendarRange(month, month, 'month'))
    # Compute the maximum precipitation for the month.
    max_precip = filtered_month.max()
    # Rename the band to 'max_precip_MM', ensuring month is formatted with two digits.
    band_name = ee.String('max_precip_').cat(ee.Number(month).format('%02d'))
    return max_precip.rename(band_name)

# Use a loop to apply this function for each month and combine the results.
max_precip_bands = [get_max_precip_for_month(month) for month in range(1, 13)]

# Initialize an empty image (without the 'constant' band issue).
combined_max_precip_image = ee.Image(max_precip_bands[0])

# Add the rest of the monthly max bands to the combined image.
for band_image in max_precip_bands[1:]:
    combined_max_precip_image = combined_max_precip_image.addBands(band_image)

# Now, `combined_max_precip_image` should have 12 bands, each named for a month's max precip.

In [8]:
stream_dist_proximity_collection = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/stream-outlet-distance/stream_dist_proximity")\
    .filterBounds(bbox)\
    .mosaic()
stream_dist_proximity = stream_dist_proximity_collection.clip(bbox).rename('stream_distance')

hydro_proj = stream_dist_proximity.projection()

## set time frame
before_start= '2023-09-25'
before_end='2023-10-05'

after_start='2023-10-05'
after_end='2023-10-15'

# SET SAR PARAMETERS (can be left default)

# Polarization (choose either "VH" or "VV")
polarization = "VH"  # or "VV"

# Pass direction (choose either "DESCENDING" or "ASCENDING")
pass_direction = "DESCENDING"  # or "ASCENDING"

# Difference threshold to be applied on the difference image (after flood - before flood)
# It has been chosen by trial and error. Adjust as needed.
difference_threshold = 1.25

# Relative orbit (optional, if you know the relative orbit for your study area)
# relative_orbit = 79

# Rename the selected geometry feature
aoi = bbox

# Load and filter Sentinel-1 GRD data by predefined parameters
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass', pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filterBounds(aoi) \
    .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine the flood result using additional datasets
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

In [9]:
# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
SLOPE = terrain.select('slope')
flooded = flooded.updateMask(SLOPE.lt(5))

# Set the default projection from the hydrography dataset
flooded = flooded.setDefaultProjection(hydro_proj)

# Now, reduce the resolution
flooded_mode = flooded.reduceResolution(
    reducer=ee.Reducer.mode(),
    maxPixels=10000
).reproject(
    crs=hydro_proj
)

In [10]:
# Create a full-area mask, initially marking everything as non-flooded (value 0)
full_area_mask = ee.Image.constant(0).clip(aoi)

# Update the mask to mark flooded areas (value 1)
# Assuming flooded_mode is a binary image with 1 for flooded areas and 0 elsewhere
flood_labeled_image = full_area_mask.where(flooded, 1)

# Now flood_labeled_image contains 1 for flooded areas and 0 for non-flooded areas

In [11]:
combined = (dem.rename("elevation")
        .addBands(landcover.select('Map').rename("landcover"))
        .addBands(slope)
        .addBands(ghsl)
        .addBands(flow_direction.rename("flow_direction"))
        .addBands(stream_dist_proximity)
        .addBands(flood_labeled_image.rename("flooded_mask"))
        .addBands(flow_accumulation)
        .addBands(spi)
        .addBands(sti)
        .addBands(cti)
        .addBands(tpi)
        .addBands(tri)
        .addBands(pcurv)
        .addBands(tcurv)
        .addBands(aspect)
        )

# Add each max_precip_per_month band individually.
# for month in range(1, 13):
   #  month_str = f"{month:02d}"  # Formats the month as a two-digit string.
    # band_name = f"max_precip_{month_str}"  # Constructs the band name, e.g., "max_precip_01".
    # combined = combined.addBands(combined_max_precip_image.select([band_name]).rename(f"max_precip_{month_str}"))


In [12]:
combined.bandNames().getInfo()

['elevation',
 'landcover',
 'slope',
 'built_characteristics',
 'flow_direction',
 'stream_distance',
 'flooded_mask',
 'flow_accumulation',
 'spi',
 'sti',
 'cti',
 'tpi',
 'tri',
 'pcurv',
 'tcurv',
 'aspect']

In [13]:
sample = landcover.sample(
  region=bbox,
  scale=10,  # Adjust as needed
  numPixels=10000,
  seed=0,
  geometries=False
)

# Extract landcover class values from the sample
# This assumes 'landcover' has a band named 'class' or similar that you're interested in
# Adjust 'class' to the name of your landcover band
sampled_values = sample.aggregate_array('Map').getInfo()

# Use Python's set for unique values
unique_values = set(sampled_values)

# Get the sampled class band values as a list
classValuesList = sample.aggregate_array('Map').getInfo()

# Calculate the histogram (frequency of each class) using Python's collections.Counter
from collections import Counter
classHistogram = Counter(classValuesList)

print(classHistogram)

Counter({10: 3458, 80: 2222, 30: 936, 40: 70, 95: 39, 50: 36, 20: 24, 90: 23, 60: 20})


In [14]:
# Get all band names from the combined image
allBandNames = combined.bandNames()

# Remove the class band name ('flooded_full_mask') to get input properties
inputProperties = allBandNames.filter(ee.Filter.neq('item', 'flooded_mask'))

totalSamples = 100000  # Target number of samples
totalSampled = sum(classHistogram.values())

# Calculate sample sizes for each class
sampleSizes = {cls: int((freq / totalSampled) * totalSamples) for cls, freq in classHistogram.items()}

# Calculate the number of samples per class
total_samples = 100000  # Total desired samples
samples_per_class = total_samples // len(unique_values)  # Evenly distribute samples

combined_samples = ee.FeatureCollection([])

# Loop through each land cover class and its calculated sample size
for landcover_class, size in sampleSizes.items():
    # Ensure the class is correctly formatted for Earth Engine
    landcover_class = int(landcover_class)
    
    # Perform stratified sampling for the filtered image
    class_stratified_sample = combined.updateMask(
        landcover.select('Map').eq(landcover_class)
    ).stratifiedSample(
        numPoints=size,
        classBand='flooded_mask',
        region=bbox,
        scale=30,
        seed=0,
        geometries=True  # Set based on need
    ).randomColumn()
    
    # Merge the samples from this class into the combined samples
    combined_samples = combined_samples.merge(class_stratified_sample)

# Randomly split the combined samples into training and testing datasets
training = combined_samples.filter(ee.Filter.lt('random', 0.7))
testing = combined_samples.filter(ee.Filter.gte('random', 0.7))

In [15]:
# Set up the Random Forest classifier for flood prediction
classifier = ee.Classifier.smileRandomForest(10).train(
    features=training,
    classProperty='flooded_mask',  # Use 'flooded_full_mask' as the class property
    inputProperties=inputProperties  # Dynamically generated input properties
)

# Classify the image
classified = combined.select(inputProperties).classify(classifier)

# Assess accuracy
testAccuracy = testing.classify(classifier).errorMatrix('flooded_mask', 'classification')

# Calculate accuracy
accuracy = testAccuracy.accuracy().getInfo()

# Convert the confusion matrix to an array
confusionMatrixArray = testAccuracy.array().getInfo()

# Calculate recall for the positive class (assuming '1' represents the positive class for flooding)
true_positives = confusionMatrixArray[1][1]  # True positives
false_negatives = confusionMatrixArray[1][0]  # False negatives
false_positives = confusionMatrixArray[0][1]  # False positives (non-flooded incorrectly identified as flooded)
true_negatives = confusionMatrixArray[0][0]  # True negatives (non-flooded correctly identified as non-flooded)
recall = true_positives / (true_positives + false_negatives)
false_positive_rate = false_positives / (false_positives + true_negatives)

print('Confusion Matrix:', confusionMatrixArray)
print('Accuracy:', accuracy)
print('Recall:', recall)
print('False Positive Rate:', false_positive_rate)

Confusion Matrix: [[29526, 452], [838, 4936]]
Accuracy: 0.9639181024837772
Recall: 0.8548666435746449
False Positive Rate: 0.015077723664020281


In [16]:
# Set up the Random Forest classifier for flood prediction with probability output
classifier = ee.Classifier.smileRandomForest(10).setOutputMode('PROBABILITY').train(
        features=training,
        classProperty='flooded_mask',
        inputProperties=inputProperties
    )

# Classify the image to get probabilities
probabilityImage = combined.select(inputProperties).classify(classifier)

# Visualization parameters for probability with white at the midpoint
vizParamsProbability = {
    'min': 0.,
    'max': 1,
    'palette': colorcet.bmw
}

# Create a binary mask based on the threshold.
floodedMask = probabilityImage.gte(0.5)  # This creates a mask where flooded areas (probability >= 0.5) are 1, others are 0.

# Visualization parameters for the binary mask, using orange for flooded areas.
vizParamsFlooded = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'orange']  # 'white' for unflooded (0), 'orange' for flooded (1).
}

In [17]:
m = geemap.Map()
m.add("basemap_selector")
m.add("layer_manager")

# Center the map on San Jose, Costa Rica
m.setCenter(-84.0833, 9.9333, 10)
# m.addLayer(landcover, {}, 'ESA WorldCover 2020')
m.addLayer(probabilityImage, vizParamsProbability, 'Flood Probability')
m.addLayer(floodedMask, vizParamsFlooded, 'Flooded Areas')
m.addLayer(swater_mask, {'palette': 'black'}, 'Permanent Surface Water')
# Display the map
m

Map(center=[9.9333, -84.0833], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…